**Camada Silver:** é responsável pela limpeza, padronização e enriquecimento dos dados, aplicando regras de negócio definidas a partir da análise exploratória realizada na camada Bronze.

Nesta etapa, os dados deixam de ser brutos e passam a apresentar consistência semântica, tipagem adequada e métricas derivadas, tornando-se aptos para consumo analítico e modelagem dimensional na camada Gold.

In [0]:
df = spark.table("workspace.games_analytics_bronze.games_march_2025")
display(df)

In [0]:
colunas_remover = ["about_the_game", "short_description", "reviews", "pct_pos_recent", "num_reviews_recent", "pct_pos_total", "peak_ccu", "average_playtime_forever", "average_playtime_2weeks", "median_playtime_forever", "median_playtime_2weeks", "score_rank", "user_score", "website", "support_url", "support_email", "packages", "screenshots", "movies", "header_image", "num_reviews_total"]

df = df.drop(*colunas_remover)

Utilizando o Apache Spark, foram removidas as colunas que não agregavam valor às análises propostas neste projeto, com o objetivo de simplificar o modelo de dados e otimizar o processamento.

In [0]:
display(df)

In [0]:
from pyspark.sql.functions import col, sum, avg, first

# 1. Filtrar jogos sem nota relevante
df = df.filter(col("metacritic_score") > 0)

# 2. Agrupar duplicados
df = df.groupBy("name").agg(
    sum("positive").alias("positive_ratings_total"),
    sum("negative").alias("negative_ratings_total"),
    avg("price").alias("price"),
    avg("metacritic_score").alias("metacritic_score"),
    first("release_date").alias("release_date"),
    first("genres").alias("genres"),
    first("developers").alias("developer"),
    first("publishers").alias("publisher"),
    first("windows").alias("windows"),
    first("mac").alias("mac"),
    first("linux").alias("linux")
)

Nesta etapa, foi realizado o agrupamento dos dados por jogo, visando a consolidação de registros duplicados existentes na base. A partir desse agrupamento, foram aplicadas funções de agregação sobre as métricas numéricas, como contagens e avaliações, assegurando a consistência e a precisão das análises posteriores.

In [0]:
display(df)

In [0]:
from pyspark.sql import functions as F
from pyspark.sql.functions import col

#Ano de Lançamento
df = df.withColumn(
    "release_year",
    col("release_date").substr(1, 4).cast("int")
)

df = df.filter((col("positive_ratings_total") > 0) & (col("negative_ratings_total") > 0))

# Taxa de aprovação (positivas / total)
df = df.withColumn(
    "total_reviews",
    F.col("positive_ratings_total") + F.col("negative_ratings_total")
)

df = df.withColumn(
    "approval_pct",
    F.round(
        (col("positive_ratings_total") / col("total_reviews")) * 100,
        2
    )
)


C = (
    df
    .select(F.avg("approval_pct").alias("avg_approval"))
    .collect()[0]["avg_approval"]
)

m = (
    df
    .selectExpr("percentile_approx(total_reviews, 0.70) as m")
    .collect()[0]["m"]
)

df = df.withColumn(
    "approval_weighted",
    F.round(
        (
            (col("total_reviews") / (col("total_reviews") + F.lit(m))) * col("approval_pct") +
            (F.lit(m) / (col("total_reviews") + F.lit(m))) * F.lit(C)
        ),
        2
    )
)

df = df.withColumn(
    "review_confidence",
    F.when(col("total_reviews") >= m, "Alta")
     .when(col("total_reviews") >= (m / 2), "Média")
     .otherwise("Baixa")
)



Foi criada uma coluna referente ao ano de lançamento dos jogos, com o objetivo de facilitar análises temporais e comparativas ao longo dos anos.
Além disso, foi desenvolvida uma coluna de aprovação total dos jogos, na qual é realizado o cálculo percentual de avaliações positivas em relação ao total de avaliações de usuários da plataforma Steam. Esse indicador tem como finalidade permitir a comparação direta com o índice de aprovação disponibilizado pelo Metacritic.

Além da taxa de aprovação simples, foi criada uma métrica de aprovação ponderada baseada no volume total de avaliações.
Essa abordagem reduz o viés estatístico de jogos com poucas reviews, combinando a nota individual com a média global da base, atribuindo maior peso conforme o número de avaliações aumenta.

In [0]:
display(df)

In [0]:
from pyspark.sql import functions as F

# 1) Limpar colchetes e criar array
df = df.withColumn(
    "genres_clean",
    F.regexp_replace(F.col("genres"), r"[\[\]]", "")
).withColumn(
    "genres_array",
    F.split(F.col("genres_clean"), ",\\s*")
)

# 2) Normalizar cada elemento: trim + lower
df = df.withColumn(
    "genres_array_clean",
    F.expr("transform(genres_array, x -> lower(trim(x)))")
)

# 3) Flag is_indie: se QUALQUER elemento contém 'indie'
df = df.withColumn(
    "is_indie",
    F.expr("exists(genres_array_clean, x -> instr(x, 'indie') > 0)")
)

# 4) Remover 'indie' do array para não tratá-lo como gênero
df = df.withColumn(
    "genres_no_indie",
    F.expr("filter(genres_array_clean, x -> instr(x, 'indie') = 0)")
)

# 5) Criar genre_1 e genre_2 a partir dos gêneros restantes

# genre_1
df = df.withColumn(
    "genre_1",
    F.when(F.size(F.col("genres_no_indie")) >= 1,
           F.element_at(F.col("genres_no_indie"), 1))
     .when(F.col("is_indie") == True, "indie")  # caso seja só indie
)

# genre_2
df = df.withColumn(
    "genre_2",
    F.when(F.size(F.col("genres_no_indie")) >= 2,
           F.element_at(F.col("genres_no_indie"), 2))
     .when(F.size(F.col("genres_no_indie")) == 1,
           F.element_at(F.col("genres_no_indie"), 1))  # duplica
     .when(F.col("is_indie") == True, "Indie")  # caso seja só indie
)

# 6) Remover colunas auxiliares
df = df.drop("genres_clean", "genres_array", "genres_array_clean", "genres_no_indie", "genres")


A coluna de gêneros (genres) passou por um processo de normalização, uma vez que os dados brutos apresentavam múltiplos valores em formato de lista. Inicialmente, foi criada uma coluna indicativa para identificar jogos classificados como Indie. Em seguida, o gênero Indie foi removido da lista principal, permitindo que os gêneros remanescentes representassem exclusivamente o estilo do jogo. Para padronização da análise, foram considerados no máximo dois gêneros principais, distribuídos em colunas separadas. Nos casos em que apenas um gênero estava disponível, o valor foi replicado, garantindo consistência estrutural dos dados.

### No contexto deste projeto, foi criada a métrica is_indie com o objetivo de:

- Diferenciar jogos independentes de produções tradicionais;

- Permitir análises comparativas entre jogos indie e não-indie;

- Avaliar se jogos indie apresentam:

- Maior aprovação dos usuários;

- Melhores avaliações relativas;

- Diferenças significativas de preço;

- Maior divergência entre opinião da crítica e dos jogadores.

### Essa classificação possibilita análises como:

- Evolução da popularidade de jogos indie ao longo do tempo;

- Participação de jogos indie entre os títulos mais bem avaliados;

- Comparação do desempenho médio entre jogos indie e jogos de grandes estúdios.

In [0]:
display(df)


In [0]:
from pyspark.sql import functions as F

df = df.withColumn(
    "publisher_clean",
    F.lower(
        F.regexp_replace(F.col("publisher"), r"[\[\]']", "")
    )
)

df = df.withColumn(
    "publisher_first",
    F.trim(F.split(F.col("publisher_clean"), ",")[0])
)

df = df.withColumn(
    "publisher_normalized",
    F.when(F.col("publisher_clean").contains("capcom"), "CAPCOM")
     .when(F.col("publisher_clean").contains("ubisoft"), "Ubisoft")
     .when(F.col("publisher_clean").contains("electronic arts"), "EA")
     .when(F.col("publisher_clean").contains("ea "), "EA")
     .when(F.col("publisher_clean").contains("square enix"), "Square Enix")
     .when(F.col("publisher_clean").contains("bandai"), "Bandai Namco")
     .when(F.col("publisher_clean").contains("sega"), "SEGA")
     .when(F.col("publisher_clean").contains("sony"), "Sony")
     .when(F.col("publisher_clean").contains("nintendo"), "Nintendo")
     .when(F.col("publisher_clean").contains("2k"), "2K")
     .when(F.col("publisher_clean").contains("bethesda"), "Bethesda")
     .when(F.col("publisher_clean").contains("activision"), "Activision")
     .when(F.col("publisher_clean").contains("konami"), "Konami")
     .when(F.col("publisher_clean").contains("thq"), "THQ")
     .when(F.col("publisher_clean").contains("team17"), "Team17")
     .when(F.col("publisher_clean").contains("telltale"), "Telltale")
     .when(F.col("publisher_clean").contains("techland"), "Techland")
     .when(F.col("publisher_clean").contains("playway"), "PlayWay")
     .when(F.col("publisher_clean").contains("microids"), "Microids")
     .when(F.col("publisher_clean").contains("kalypso"), "Kalypso Media")
     .when(F.col("publisher_clean").contains("klei"), "Klei Entertainment")
     .when(F.col("publisher_clean").contains("frontier"), "Frontier Developments")
     .when(F.col("publisher_clean").contains("double fine"), "Double Fine Productions")
     .when(F.col("publisher_clean").contains("disney"), "Disney")
     .when(F.col("publisher_clean").contains("warner"), "Warner Bros. Games")
     .when(F.col("publisher_clean").contains("viva media"), "Viva Media")
     .when(F.col("publisher_clean").contains("wargaming"), "Wargaming")
     .when(F.col("publisher_clean").contains("505 games"), "505 Games")
     

     
    
     .otherwise(F.initcap(F.col("publisher_first")))
)

df = df.drop("publisher_clean", "publisher_first", "publisher")



Foi realizada uma normalização das publishers para evitar fragmentação de dados causada por variações de nomenclatura. Caso a publisher não se enquadre nas regras conhecidas, foi adotada a primeira ocorrência da lista como valor padrão.

In [0]:
display(df)

In [0]:
from pyspark.sql import functions as F

df = df.withColumn(
    "developer_clean",
    F.lower(
        F.regexp_replace(F.col("developer"), r"[\[\]']", "")
    )
)

df = df.withColumn(
    "developer_first",
    F.trim(F.split(F.col("developer_clean"), ",")[0])
)


df = df.withColumn(
    "developer_normalized",
    F.when(F.col("developer_clean").contains("capcom"), "CAPCOM")
     .when(F.col("developer_clean").contains("ubisoft"), "Ubisoft")
     .when(F.col("developer_clean").contains("1c"), "1C Game Studio")
     .when(F.col("developer_clean").contains("2k"), "2K")
     .when(F.col("developer_clean").contains("505 games"), "505 Games")
     .when(F.col("developer_clean").contains("activision"), "Activision")
     .when(F.col("developer_clean").contains("animation arts"), "Animation Arts")
     .when(F.col("developer_clean").contains("bandai"), "Bandai Namco")     
     .when(F.col("developer_clean").contains("bethesda"), "Bethesda Game Studio")
     .when(F.col("developer_clean").contains("bioware"), "BioWare")
     .when(F.col("developer_clean").contains("capybara"), "Capybara Games")
     .when(F.col("developer_clean").contains("coreplay"), "Coreplay")
     .when(F.col("developer_clean").contains("bandai"), "Bandai Namco")
     .when(F.col("developer_clean").contains("creative assembly"), "Creative Assembly")
     .when(F.col("developer_clean").contains("cyanide"), "Cyanide Studios")
     .when(F.col("developer_clean").contains("cyberconnect2"), "CyberConnect2")
     .when(F.col("developer_clean").contains("deck13"), "Deck13")
     .when(F.col("developer_clean").contains("cyberconnect2"), "CyberConnect2")
     .when(F.col("developer_clean").contains("deep silver"), "Deep Silver")
     .when(F.col("developer_clean").contains("cyberconnect2"), "CyberConnect2")
     .when(F.col("developer_clean").contains("ea"), "EA")
     .when(F.col("developer_clean").contains("eletronic arts"), "EA")
     .when(F.col("developer_clean").contains("eidos"), "Eidos")
     .when(F.col("developer_clean").contains("gaming minds"), "Gaming Minds")
     .when(F.col("developer_clean").contains("konami"), "Konami")
     .when(F.col("developer_clean").contains("rockstar"), "Rockstar Games")
     .when(F.col("developer_clean").contains("square enix"), "Square Enix")
     .when(F.col("developer_clean").contains("thq"), "THQ")
     .when(F.col("developer_clean").contains("eidos"), "Eidos")
     .when(F.col("developer_clean").contains("team17"), "Team17")
     .when(F.col("developer_clean").contains("telltale"), "Telltale")
     .when(F.col("developer_clean").contains("team17"), "Team17")
     .when(F.col("developer_clean").contains("vertigo"), "Vertigo Games")


     .otherwise(F.initcap(F.col("developer_first")))
    
)

df = df.drop("developer", "developer_clean", "developer_first")

In [0]:
display(df)

Foi realizada uma normalização dos developers para evitar fragmentação de dados causada por variações de nomenclatura. Caso o developer não se enquadre nas regras conhecidas, foi adotada a primeira ocorrência da lista como valor padrão.

In [0]:
%sql
DROP TABLE IF EXISTS workspace.games_analytics_silver.games_march_2025;


In [0]:
df.write \
  .format("delta") \
  .mode("overwrite") \
  .saveAsTable("workspace.games_analytics_silver.games_march_2025")


Carga dos dados tratados na tabela Silver.

Ao final da camada Silver, os dados foram submetidos a um processo de limpeza, padronização e enriquecimento, garantindo maior qualidade e confiabilidade analítica.

Foram aplicadas transformações voltadas à normalização de campos textuais, tratamento de dados categóricos, criação de métricas derivadas e padronização de tipos, além da definição de regras de negócio para avaliações de usuários e classificação de jogos.

Com isso, a base resultante apresenta estrutura consistente e semântica clara, estando preparada para a modelagem dimensional e construção da camada Gold, onde os dados passam a ser organizados para consumo analítico e visualização.